#**ChurnNet For Telecom Clustomer Churn**

##Import thư viện

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Add
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Conv1D, Activation, Multiply

In [2]:
df_raw = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn - Copy (2).csv")

In [3]:
df_raw.shape

(7043, 21)

## Preprocessing

In [4]:
df_raw.loc[df_raw["MultipleLines"] == "No phone service", "MultipleLines"] = 'No'
df_raw.loc[df_raw["OnlineSecurity"] == "No internet service", "OnlineSecurity"] = 'No'
df_raw.loc[df_raw["OnlineBackup"] == "No internet service", "OnlineBackup"] = 'No'
df_raw.loc[df_raw["DeviceProtection"] == "No internet service", "DeviceProtection"] = 'No'
df_raw.loc[df_raw["TechSupport"] == "No internet service", "TechSupport"] = 'No'
df_raw.loc[df_raw["StreamingTV"] == "No internet service", "StreamingTV"] = 'No'
df_raw.loc[df_raw["StreamingMovies"] == "No internet service", "StreamingMovies"] = 'No'
df_raw.loc[df_raw["DeviceProtection"] == "No internet service", "DeviceProtection"] = 'No'
df_raw.loc[df_raw["TotalCharges"] == " ", "TotalCharges"] = '0'
df_raw['TotalCharges'] = pd.to_numeric(df_raw['TotalCharges'])

In [5]:
# Encoder = "Label Encoder"
Encoder = "Label Encoder"
# OverSamplingTecnique = ""
OverSamplingTecnique = "SMOTE-Enn"
# OverSamplingTecnique = "SMOTE-Tomek"
# OverSamplingTecnique = "SMOTE-Enn"
filter_size=3
number_of_filter=128
flatten_layer_exist=True
Model_Name="SE Block"

##Label Encoding

In [7]:
import joblib
if Encoder == "Label Encoder":
  print("Applying Label Encoder")
  df_final = df_raw.copy()
  le = LabelEncoder()

  text_data_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
              'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
              'StreamingTV', 'StreamingMovies', 'Contract','PaperlessBilling', 'PaymentMethod', 'Churn']

  print('Label Encoder Transformation')
  for i in text_data_features :
      df_final[i] = le.fit_transform(df_final[i])
      print(i,' : ',df_final[i].unique(),' = ',le.inverse_transform(df_final[i].unique()))
  # Lưu encoder
  joblib.dump(le, "label_encoder.pkl")


  X = df_final.drop(['Churn','customerID'], axis=1).copy()
  Y = df_final['Churn'].copy().astype(int)

Applying Label Encoder
Label Encoder Transformation
gender  :  [0 1]  =  ['Female' 'Male']
SeniorCitizen  :  [0 1]  =  [0 1]
Partner  :  [1 0]  =  ['Yes' 'No']
Dependents  :  [0 1]  =  ['No' 'Yes']
PhoneService  :  [0 1]  =  ['No' 'Yes']
MultipleLines  :  [0 1]  =  ['No' 'Yes']
InternetService  :  [0 1 2]  =  ['DSL' 'Fiber optic' 'No']
OnlineSecurity  :  [0 1]  =  ['No' 'Yes']
OnlineBackup  :  [1 0]  =  ['Yes' 'No']
DeviceProtection  :  [0 1]  =  ['No' 'Yes']
TechSupport  :  [0 1]  =  ['No' 'Yes']
StreamingTV  :  [0 1]  =  ['No' 'Yes']
StreamingMovies  :  [0 1]  =  ['No' 'Yes']
Contract  :  [0 1 2]  =  ['Month-to-month' 'One year' 'Two year']
PaperlessBilling  :  [1 0]  =  ['Yes' 'No']
PaymentMethod  :  [2 3 0 1]  =  ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn  :  [0 1]  =  ['No' 'Yes']


**SMOTEENN**

In [8]:
if OverSamplingTecnique == "SMOTE-Enn":
  print("Applying SMOTE-Enn")

  smote_enn = SMOTEENN()
  X_resampled, y_resampled = smote_enn.fit_resample(X, Y)
  scaler = StandardScaler()
  X_resampled_scaled = scaler.fit_transform(X_resampled)
  X=X_resampled_scaled
  Y=y_resampled
  joblib.dump(scaler, "scaler.pkl")

Applying SMOTE-Enn


##Squeeze-and-Excitation

In [9]:
# Define the channel attention layer
class ChannelAttention(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=8,**kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        channels = input_shape[-1]
        self.fc = tf.keras.layers.Dense(channels // self.reduction_ratio, activation='relu')
        self.attention = tf.keras.layers.Dense(channels, activation='sigmoid')

    def call(self, inputs):
        x = tf.reduce_mean(inputs, axis=[1])  # Global average pooling across time dimension
        x = self.fc(x)
        x = self.attention(x)
        x = tf.expand_dims(x, axis=1)  # Add a new dimension for broadcasting
        return inputs * x

# Define the spatial attention layer
class SpatialAttention(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.max_pool = tf.keras.layers.MaxPooling1D(pool_size=3, strides=1, padding='same')
        self.avg_pool = tf.keras.layers.AveragePooling1D(pool_size=3, strides=1, padding='same')
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        self.conv1d = tf.keras.layers.Conv1D(filters=1, kernel_size=3, padding='same', activation='sigmoid')

    def call(self, inputs):
        max_pool_out = self.max_pool(inputs)
        avg_pool_out = self.avg_pool(inputs)
        concat_out = self.concat([max_pool_out, avg_pool_out])
        attention_weights = self.conv1d(concat_out)
        return inputs * attention_weights


# Define the residual block
def residual_block(x, filters, kernel_size):
    # Save the input tensor
    x_shortcut = x

    # First convolutional layer
    x = tf.keras.layers.Conv1D(filters, kernel_size, activation='relu', padding='same')(x)

    # Second convolutional layer
    x = tf.keras.layers.Conv1D(filters, kernel_size, activation='relu', padding='same')(x)

    # Add the shortcut connection
    x = Add()([x, x_shortcut])

    # Apply ReLU activation
    x = tf.keras.layers.Activation('relu')(x)

    return x

##Basic Channel Attention

In [10]:
# Define the channel attention layer for 1D data
class Basic_ChannelAttention(tf.keras.layers.Layer):
    def __init__(self, ratio=8,**kwargs):
        super(Basic_ChannelAttention, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        _, channels = input_shape[1:]
        self.shared_layer1 = Conv1D(channels // self.ratio, kernel_size=1, activation='relu', padding='same')
        self.shared_layer2 = Conv1D(channels, kernel_size=1, padding='same')

    def call(self, inputs):
        x1 = tf.reduce_mean(inputs, axis=1, keepdims=True)
        x1 = self.shared_layer1(x1)
        x1 = self.shared_layer2(x1)

        x2 = tf.reduce_max(inputs, axis=1, keepdims=True)
        x2 = self.shared_layer1(x2)
        x2 = self.shared_layer2(x2)

        attention = tf.add(x1, x2)
        attention = Activation("sigmoid")(attention)
        output = Multiply()([inputs, attention])

        return output

##**Note: Cell này và cell dưới không phục vụ Web API, chỉ Run khi bạn muốn xem kết quả mô hình**

In [21]:
def kfold(filter_size, number_of_filter, flatten_layer_exist, Model_Name):
    print("Applying K-fold")
    print(f"Applying {number_of_filter} filters of size {filter_size}")

    num_folds = 5  # Đã giảm xuống 5

    # Các biến lưu kết quả
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    mcc_scores = []
    auc_roc_scores = []

    skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
    fold_number = 1

    for train_index, test_index in skf.split(X, Y):
        print(f"\nFold {fold_number}/{num_folds}:")
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        # Kiến trúc model
        inputs = tf.keras.Input(shape=(X_train.shape[1], 1))
        x = tf.keras.layers.Conv1D(filters=number_of_filter, kernel_size=filter_size, activation='relu')(inputs)
        x_res = residual_block(x, number_of_filter, filter_size)

        if Model_Name == "SE Block":
            print("Applying SE Block")
            x = ChannelAttention()(x_res)
        else:
            print("Applying Basic Channel Attention")
            x = Basic_ChannelAttention()(x_res)

        x = SpatialAttention()(x)
        x = tf.keras.layers.Conv1D(filters=number_of_filter, kernel_size=filter_size, activation='relu', padding='same')(x)
        x_res = residual_block(x, number_of_filter, filter_size)

        if Model_Name == "SE Block":
            x = ChannelAttention()(x_res)
        else:
            x = Basic_ChannelAttention()(x_res)

        x = SpatialAttention()(x)

        if flatten_layer_exist:
            print("Applying Flatten layer")
            x = tf.keras.layers.Flatten()(x)
        else:
            print("Applying Global Max Pooling 1D layer")
            x = tf.keras.layers.GlobalMaxPooling1D()(x)

        x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.Dense(number_of_filter, activation='relu')(x)
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

        model.fit(X_train, Y_train, epochs=30, batch_size=32, verbose=1, validation_split=0.2)

        # Lưu mô hình sau mỗi fold
        model_save_path = f"model_fold_{fold_number}.h5"
        model.save(model_save_path)
        print(f"✅ Saved model to: {model_save_path}")

        # Đánh giá
        Y_pred = model.predict(X_test)
        Y_pred_binary = np.round(Y_pred).flatten()

        accuracy = accuracy_score(Y_test, Y_pred_binary)
        precision = precision_score(Y_test, Y_pred_binary)
        recall = recall_score(Y_test, Y_pred_binary)
        f1 = f1_score(Y_test, Y_pred_binary)
        mcc = matthews_corrcoef(Y_test, Y_pred_binary)
        auc_roc = roc_auc_score(Y_test, Y_pred)

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        mcc_scores.append(mcc)
        auc_roc_scores.append(auc_roc)

        print(f"✅ Fold {fold_number} Results:")
        print(f"  - Accuracy : {accuracy:.4f}")
        print(f"  - Precision: {precision:.4f}")
        print(f"  - Recall   : {recall:.4f}")
        print(f"  - F1 Score : {f1:.4f}")
        print(f"  - MCC      : {mcc:.4f}")
        print(f"  - AUC-ROC  : {auc_roc:.4f}")
        fold_number += 1
        print("Kiểu object:", type(X_test))
        print("Kiểu dữ liệu mỗi phần tử:", X_test.dtype)
        print("Shape:", X_test.shape)


In [22]:
kfold(filter_size, number_of_filter, flatten_layer_exist, Model_Name)

Applying K-fold
Applying 128 filters of size 3

Fold 1/5:
Applying SE Block
Applying Flatten layer
Epoch 1/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.7503 - loss: 0.4698 - val_accuracy: 0.9543 - val_loss: 0.1922
Epoch 2/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9068 - loss: 0.2409 - val_accuracy: 0.9372 - val_loss: 0.1871
Epoch 3/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9072 - loss: 0.2273 - val_accuracy: 0.9489 - val_loss: 0.1548
Epoch 4/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9060 - loss: 0.2241 - val_accuracy: 0.9798 - val_loss: 0.0620
Epoch 5/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9146 - loss: 0.2176 - val_accuracy: 0.8809 - val_loss: 0.3084
Epoch 6/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9124 - loss: 0.2151 - val_accuracy: 0.9670 - val_loss: 0.1416
Epoch 7/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9193 - loss: 0.2002 - val_accuracy: 0.9372 - val_loss: 0.1667
Epo

✅ Saved model to: model_fold_1.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
✅ Fold 1 Results:
  - Accuracy : 0.9199
  - Precision: 0.9328
  - Recall   : 0.9181
  - F1 Score : 0.9254
  - MCC      : 0.8391
  - AUC-ROC  : 0.9797
Kiểu object: <class 'numpy.ndarray'>
Kiểu dữ liệu mỗi phần tử: float64
Shape: (1174, 19)

Fold 2/5:
Applying SE Block
Applying Flatten layer
Epoch 1/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.7821 - loss: 0.4513 - val_accuracy: 0.8968 - val_loss: 0.2785
Epoch 2/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9003 - loss: 0.2518 - val_accuracy: 0.9223 - val_loss: 0.2051
Epoch 3/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9159 - loss: 0.2126 - val_accuracy: 0.9628 - val_loss: 0.1258
Epoch 4/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9115 - loss: 0.2209 - val_accuracy: 0.9149 - val_loss: 0.1950
Epoch 5/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9164 - loss: 0.2139 - val_accuracy: 0.9011 - val_loss: 0

✅ Saved model to: model_fold_2.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
✅ Fold 2 Results:
  - Accuracy : 0.9174
  - Precision: 0.9256
  - Recall   : 0.9213
  - F1 Score : 0.9234
  - MCC      : 0.8337
  - AUC-ROC  : 0.9752
Kiểu object: <class 'numpy.ndarray'>
Kiểu dữ liệu mỗi phần tử: float64
Shape: (1174, 19)

Fold 3/5:
Applying SE Block
Applying Flatten layer
Epoch 1/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.7565 - loss: 0.4586 - val_accuracy: 0.9117 - val_loss: 0.2132
Epoch 2/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8923 - loss: 0.2643 - val_accuracy: 0.9330 - val_loss: 0.1724
Epoch 3/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9065 - loss: 0.2329 - val_accuracy: 0.9691 - val_loss: 0.1492
Epoch 4/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9075 - loss: 0.2301 - val_accuracy: 0.9372 - val_loss: 0.1478
Epoch 5/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9119 - loss: 0.2205 - val_accuracy: 0.9585 - val_loss: 0

✅ Saved model to: model_fold_3.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
✅ Fold 3 Results:
  - Accuracy : 0.9242
  - Precision: 0.9361
  - Recall   : 0.9228
  - F1 Score : 0.9294
  - MCC      : 0.8477
  - AUC-ROC  : 0.9777
Kiểu object: <class 'numpy.ndarray'>
Kiểu dữ liệu mỗi phần tử: float64
Shape: (1174, 19)

Fold 4/5:
Applying SE Block
Applying Flatten layer
Epoch 1/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.7973 - loss: 0.4780 - val_accuracy: 0.9649 - val_loss: 0.1263
Epoch 2/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8973 - loss: 0.2411 - val_accuracy: 0.9191 - val_loss: 0.1970
Epoch 3/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9041 - loss: 0.2255 - val_accuracy: 0.9287 - val_loss: 0.1813
Epoch 4/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9089 - loss: 0.2272 - val_accuracy: 0.9606 - val_loss: 0.1010
Epoch 5/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9177 - loss: 0.2106 - val_accuracy: 0.9394 - val_loss: 0

✅ Saved model to: model_fold_4.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
✅ Fold 4 Results:
  - Accuracy : 0.9259
  - Precision: 0.9349
  - Recall   : 0.9276
  - F1 Score : 0.9312
  - MCC      : 0.8509
  - AUC-ROC  : 0.9744
Kiểu object: <class 'numpy.ndarray'>
Kiểu dữ liệu mỗi phần tử: float64
Shape: (1174, 19)

Fold 5/5:
Applying SE Block
Applying Flatten layer
Epoch 1/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.7565 - loss: 0.4746 - val_accuracy: 0.8883 - val_loss: 0.3184
Epoch 2/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8982 - loss: 0.2365 - val_accuracy: 0.9489 - val_loss: 0.1459
Epoch 3/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9078 - loss: 0.2350 - val_accuracy: 0.9851 - val_loss: 0.0605
Epoch 4/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9048 - loss: 0.2211 - val_accuracy: 0.9415 - val_loss: 0.1414
Epoch 5/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9127 - loss: 0.2123 - val_accuracy: 0.9191 - val_loss: 0

✅ Saved model to: model_fold_5.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step
✅ Fold 5 Results:
  - Accuracy : 0.9344
  - Precision: 0.9589
  - Recall   : 0.9182
  - F1 Score : 0.9382
  - MCC      : 0.8693
  - AUC-ROC  : 0.9823
Kiểu object: <class 'numpy.ndarray'>
Kiểu dữ liệu mỗi phần tử: float64
Shape: (1174, 19)


In [14]:
import gradio as gr
import numpy as np
import json
import joblib
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
scaler = joblib.load("/content/scaler (1).pkl")

# =============================
# Load scaler (dùng sau SMOTE-ENN)
# =============================
feature_columns = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges']
int_columns = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod']


# =============================
# Load ChurnNet model (có custom layers)
# =============================

churnet_model = load_model("/content/model_fold_3 (1).h5", custom_objects={
    "ChannelAttention": ChannelAttention,
    "Basic_ChannelAttention": Basic_ChannelAttention,
    "SpatialAttention": SpatialAttention
})

# =============================
# Load các model truyền thống
# =============================
models = {
    "Logistic Regression": joblib.load("/content/logistic_regression_model.pkl"),
    "Random Forest": joblib.load("/content/random_forest_model.pkl"),
    "XGBoost": joblib.load("/content/gradient_boosting_model.pkl"),
    "ChurnNet": churnet_model
}

# =============================
# Load metric đánh giá
# =============================
with open("/content/metrics.json", "r") as f:
    metrics = json.load(f)

model_map = {
    "Logistic Regression": "logistic",
    "Random Forest": "rf",
    "XGBoost": "xgb",
    "ChurnNet": "churnnet"
}

# =============================
# Hàm tiền xử lý đầu vào
# =============================

import pandas as pd


def preprocess_input(raw_features, model_name):
    X_np = np.array([raw_features], dtype=np.float64)

    if model_name != "ChurnNet":
        # Không dùng scaler cho các mô hình truyền thống
        X_df = pd.DataFrame(X_np, columns=feature_columns)
        X_df[int_columns] = X_df[int_columns].astype(int)
        return X_df
    else:
        X_scaled = scaler.transform(X_np)  # ✅ chỉ dùng scaler đã fit trước đó
        return X_scaled  # Trả về numpy array


# =============================
# Hàm dự đoán
def predict_churn(model_name, *features):
    model = models[model_name]
    X_input = preprocess_input(features, model_name)

    if model_name == "ChurnNet":
        prob = model.predict(X_input)[0][0]   # dùng flatten để tránh lỗi shape
        prediction = 1 if prob >= 0.5 else 0
    else:
        prediction = model.predict(X_input)[0]

    label = "Churn" if prediction == 1 else "Not Churn"
    m = metrics[model_map[model_name]]
    metrics_str = (
        f"**AUC**: {m['AUC']}\n"
        f"**Accuracy**: {m['Accuracy']}\n"
        f"**Recall**: {m['Recall']}\n"
        f"**Precision**: {m['Precision']}"
    )

    return label, metrics_str


# =============================
# Gradio UI
# =============================
with gr.Blocks(title="Churn Predictor (Mixed Models)") as demo:
    gr.Markdown("## 🔮 Dự đoán khách hàng Churn từ 4 mô hình")

    with gr.Row():
        model_select = gr.Dropdown(choices=list(models.keys()), label="Chọn mô hình")

    with gr.Column():
        gender = gr.Dropdown(choices=[0, 1], label="Giới tính", value=0)
        senior = gr.Dropdown(choices=[0, 1], label="SeniorCitizen", value=0)
        partner = gr.Dropdown(choices=[0, 1], label="Partner", value=0)
        dependents = gr.Dropdown(choices=[0, 1], label="Dependents", value=0)
        tenure = gr.Number(label="Tenure")
        phone = gr.Dropdown(choices=[0, 1], label="Phone Service", value=1)
        multiple_lines = gr.Dropdown(choices=[0, 1], label="Multiple Lines", value=0)
        internet = gr.Dropdown(choices=[0, 1, 2], label="Internet Service (0: DSL, 1: Fiber Optic, 2: Không)", value=0)
        online_sec = gr.Dropdown(choices=[0, 1], label="Online Security", value=0)
        online_backup = gr.Dropdown(choices=[0, 1], label="Online Backup", value=0)
        device_protect = gr.Dropdown(choices=[0, 1], label="Device Protection", value=0)
        tech_support = gr.Dropdown(choices=[0, 1], label="Tech Support", value=0)
        streaming_tv = gr.Dropdown(choices=[0, 1], label="Streaming TV", value=0)
        streaming_movies = gr.Dropdown(choices=[0, 1], label="Streaming Movies", value=0)
        contract = gr.Dropdown(choices=[0, 1, 2], label="Contract (0: Month-to-month, 1: One year, 2: Two year)", value=0)
        paperless = gr.Dropdown(choices=[0, 1], label="Paperless Billing", value=1)
        payment_method = gr.Dropdown(choices=[0, 1, 2, 3], label="Payment Method (0: Bank Transfer, 1: Credit Card, 2-3: Check)", value=0)
        monthly_charges = gr.Number(label="Monthly Charges")
        total_charges = gr.Number(label="Total Charges")



    predict_btn = gr.Button("🔍 Predict")

    with gr.Row():
        result_label = gr.Textbox(label="Kết quả dự đoán", interactive=False)
        metrics_display = gr.Markdown()

    predict_btn.click(
        predict_churn,
        inputs=[
            model_select, gender, senior, partner, dependents, tenure, phone,
            multiple_lines, internet, online_sec, online_backup, device_protect,
            tech_support, streaming_tv, streaming_movies, contract, paperless,
            payment_method, monthly_charges, total_charges
        ],
        outputs=[result_label, metrics_display]
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://70d22e3d2176bdb14a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
